### DATOS

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import random


# Informacion del sistema
#Barra Tipo Vmag Pdem   Qdem   Pg-min Pg-max    a      b       c

G=[[1, 2, 1.010, 00.000, 00.500, 10.000, 500.00, 100.0, 35.50, 1.2500],
   [2, 2, 1.000, 000.00, 00.000, 20.000, 100.00, 25.00, 10.50, 0.5000],
   [3, 2, 1.000, 45.000, 24.000, 25.000, 150.00, 30.00, 15.50, 0.7500],
   [4, 0, 1.025, 60.000, 36.000, 00.000, 00.000, 00.00, 0.000, 0.0000],
   [5, 0, 1.000, 75.000, 48.000, 00.000, 00.000, 00.00, 0.000, 0.0000],
   [6, 2, 1.000, 90.000, 60.000, 00.000, 00.000, 00.00, 0.000, 0.0000]]




# Informacion del sistema de transmision
# Inicio Final R(p.u.) X(p.u.) Smax(MW)

B=[ [1, 2, 0.000, 0.1000, 120.00*1.5],
    [1, 3, 0.000, 0.0500, 60.000*1.5],
    [1, 4, 0.000, 0.1250, 150.00*1.5],
    [2, 4, 0.000, 0.0500, 60.000*1.5],
    [2, 5, 0.000, 0.2000, 240.00*1.5],
    [3, 5, 0.000, 0.0500, 60.000*1.5],
    [3, 6, 0.000, 0.1250, 150.00*1.5],
    [4, 6, 0.000, 0.0500, 60.000*1.5],
    [5, 6, 0.000, 0.100, 120.00*1.5]]

# HIDRO
#   Qmax(m3/h)  horas precio ($/m3)
H = [[005.000, 06.00, 010.000],
     [010.000, 08.00, 050.000],
     [003.000, 10.00, 005.000]]

#   barra
Gh =[6]

n_bloques = 3
max_caudal = [150] # m3/dia
conver_potencia = 8

nB = len(G) # número de barras
nL = len(B) # número de líneas
Sb = 100

### GENÉTICO FLUJO DC

In [9]:

generaciones = 30000
papis = []
y_papis =[]

y_papis_bloques = []

n = 100
costos = []

nB = len(G) #número de barras
nL = len(B) #número de líneas


# Índices de barras
termicas_indices = [i for i in range(nB) if G[i][1] == 2 and G[i][0] not in Gh]
idx_hidro = [bar - 1 for bar in Gh]





#penalizacion por superacion de limites de lineas
gran_penalizacion = 1e3




Yb = np.zeros((nB, nB),dtype=float) #matriz de admitancias 

# Matriz de admitancias
for i in range(nL):
    Yb[B[i][0]-1][B[i][1]-1] = -1/(B[i][3])
    Yb[B[i][1]-1][B[i][0]-1] = -1/(B[i][3])

for i in range(len(Yb)):
    Yb[i][i]=-sum(Yb[i][:])

# coeficientes de costo
c = np.array([G[j][9] for j in range(nB)])
b = np.array([G[j][8] for j in range(nB)])
a = np.array([G[j][7] for j in range(nB)])

# Carga total del sistema
demanda= sum([G[i][3] for i in range(nB)])
demanda_barras = np.array([G[i][3] for i in range(nB)])


while len(papis) < 2*n:
    posi = np.zeros((n_bloques,nB), dtype=float)
    for k in range(n_bloques):
        for j in range(nB):
            if G[j][1] == 2 :
                if G[j][0] in Gh:
                    caudal = random.uniform(0,H[k][0])
                    posi[k][j] = caudal * conver_potencia
                else:
                    posi[k][j] = random.uniform(G[j][5], G[j][6])
        
            else:
                posi[k][j] = 0.0


    # Normalizar las generaciones
    suma_generaciones = np.sum(posi,axis=1) 
    factores = np.array([demanda/suma_generaciones[k] for k in range(3)])

    posi = posi* factores[:, np.newaxis] 


    # Validación 
    validar_generacion_flujo = True
    agua_total = 0
    if not np.all([G[j][5] <= posi[k, j] <= G[j][6] for k in range(n_bloques) for j in termicas_indices]):
        validar_generacion_flujo = False

    for k in range(n_bloques):
        for barra_hidro in Gh:
            idx = barra_hidro - 1
            caudal_hidro = posi[k, idx] / conver_potencia
            agua_total += caudal_hidro * H[k][1]
            if caudal_hidro> H[k][0]:
                validar_generacion_flujo = False
                break  # Romper el bucle de barras hidro
        if not validar_generacion_flujo:
            break  # Romper el bucle de bloques

    
    if agua_total > max_caudal[0]:
        validar_generacion_flujo = False

    # Solucion de flujo DC   

    costo_penalizacion = np.zeros((3,1), dtype=float)
    if validar_generacion_flujo:

        for k in range(n_bloques):
            try:
                xp = (posi[k,:] - demanda_barras) / Sb
                deltas = np.linalg.solve(Yb[1:, 1:], xp[1:])
                deltas = np.insert(deltas, 0, 0)
                S = np.zeros(nL, dtype=float)
                for l in range(nL):
                    S[l] = abs(deltas[B[l][0]-1] - deltas[B[l][1]-1]) / B[l][3]
                    S[l] *= Sb
                    if S[l] > B[l][4]: #si supera el limite de la linea
                        #penalizar
                        costo_penalizacion[k][0] += (S[l] - B[l][4]) * gran_penalizacion
            except np.linalg.LinAlgError:
                validar_generacion_flujo = False
                break
    
    # costos
    if validar_generacion_flujo:
        costo_total= 0
        costo_despacho = np.zeros((n_bloques), dtype=float)
        for i in range(n_bloques):
            potencias_bloque = posi[i, :]
    
            # Costos
            costo_hidro_bloque = 0
            costo_termicas_bloque = 0
            
            # Costo hidroseléctricas
            for barra_hidro in Gh:
                caudal = potencias_bloque[barra_hidro - 1] / conver_potencia
                costo_hidro_bloque += caudal * H[i][2]

                
            # Costo térmicas
            costo_termicas_bloque = np.sum(c[j] * (potencias_bloque[j]**2) + b[j] * potencias_bloque[j] + a[j] for j in termicas_indices)
            
            # Sumar costos del bloque al total
            costo_despacho[i] = costo_termicas_bloque + costo_hidro_bloque + costo_penalizacion[i][0]
            costo_total += (costo_termicas_bloque + costo_hidro_bloque) * H[i][1] + (costo_penalizacion[i][0] * H[i][1])
        
        y_papis.append(costo_total)
        y_papis_bloques.append(costo_despacho)
        papis.append(posi)

print(f"Población inicial generada con éxito.{len(papis)} soluciones válidas.")

mejorcito = np.argmin(y_papis)
print(f"Mejor solución inicial encontrada con costo: {papis[mejorcito]}")

for i in range(generaciones):
    # Elige al mejor padre y los mejores de la población
    index_mej_y_papi = np.argmin(y_papis)
    mejor_papi = papis[index_mej_y_papi]
    mejor_y_papi = y_papis[index_mej_y_papi]
    
    mejores_indices = np.argsort(y_papis)[:n]
    papis_mejores = [papis[k] for k in mejores_indices]

    papis_nuevos = []
    y_papis_nuevos = []
    
    # Agrega el mejor padre a la nueva población para no perderlo
    papis_nuevos.append(mejor_papi)
    y_papis_nuevos.append(mejor_y_papi)
    costos.append(mejor_y_papi)
    
    # Generar la nueva población de hijos
    while len(papis_nuevos) < 2 * n:
        # Elige 2 padres de forma aleatoria
        l1 = random.choice(papis_mejores)
        l2 = random.choice(papis_mejores)
        lbd = np.random.uniform(0.3, 0.7)
        
        # Crossover: Crea un hijo combinando las matrices de los padres
        hijo = np.array(l1 * lbd + l2 * (1 - lbd))

        # Mutación: Selecciona un bloque y una barra al azar para mutar
        if random.random() < 0.1: 
            bloque_mutacion = random.randint(0, n_bloques - 1)
            gen = random.choice([j for j in range(nB) if G[j][1] == 2])
            
            # Ajusta el valor del generador en el bloque seleccionado
            if G[gen][0] in Gh:
                caudal = random.uniform(0, H[bloque_mutacion][0])
                hijo[bloque_mutacion, gen] = caudal * conver_potencia
            else:
                hijo[bloque_mutacion, gen] = random.uniform(G[gen][5], G[gen][6])


        # Normalizar las generaciones
        suma_generaciones = np.sum(hijo,axis=1) 
        factores = np.array([demanda/suma_generaciones[k] for k in range(n_bloques)])

        hijo = hijo* factores[:, np.newaxis] 


        # Validación 
        validar_hijo = True
        agua_total = 0
        if not np.all([G[j][5] <= hijo[k, j] <= G[j][6] for k in range(n_bloques) for j in termicas_indices]):
            validar_hijo = False


        for k in range(n_bloques):
            for barra_hidro in Gh:
                idx = barra_hidro - 1
                caudal_hidro = hijo[k, idx] / conver_potencia
                agua_total += caudal_hidro * H[k][1]
                if caudal_hidro> H[k][0]:
                    validar_hijo = False
                    break  # Romper el bucle de barras hidro
            if not validar_hijo:
                break  # Romper el bucle de bloques

            
        if agua_total > max_caudal[0]:
            validar_hijo = False


        # Flujo de carga DC
        costo_penalizacion = np.zeros((3,1), dtype=float)
        if validar_hijo:
            for k in range(n_bloques):
                try:
                    xp = (hijo[k, :] - demanda_barras) / Sb
                    deltas = np.linalg.solve(Yb[1:, 1:], xp[1:])
                    deltas = np.insert(deltas, 0, 0)
                    S = np.zeros(nL, dtype=float)
                    for l in range(nL):
                        S[l] = abs(deltas[B[l][0]-1] - deltas[B[l][1]-1]) / B[l][3]
                        S[l] *= Sb
                        if S[l] > B[l][4]:
                            costo_penalizacion[k][0] += (S[l] - B[l][4]) * gran_penalizacion
                except np.linalg.LinAlgError:
                    validar_hijo = False
                    break
        

        # Si el hijo es válido, calcula su costo y lo agrega a la nueva población
        if validar_hijo:
            costo_total = 0
            
            for k in range(n_bloques):
                potencias_bloque = hijo[k, :]
                
                # Costo de generadores térmicos
                costo_termicas = np.sum(c[j] * (potencias_bloque[j]**2) + b[j] * potencias_bloque[j] + a[j] for j in termicas_indices)
                
                # Costo y caudal de generadores hidroeléctricos (versión general)
                costo_hidro_bloque = 0
                for barra_hidro in Gh:
                    caudal = potencias_bloque[barra_hidro - 1] / conver_potencia
                    costo_hidro_bloque += caudal * H[k][2]

                    
                costo_total += (costo_termicas + costo_hidro_bloque) * H[k][1] + costo_penalizacion[k][0] * H[k][1]

            papis_nuevos.append(hijo)
            y_papis_nuevos.append(costo_total)

    papis = papis_nuevos
    y_papis = y_papis_nuevos



C:\Users\Asus\AppData\Local\Temp\ipykernel_14828\1538967867.py:130: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_14828\1538967867.py:242: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.



Población inicial generada con éxito.200 soluciones válidas.
Mejor solución inicial encontrada con costo: [[109.53139993  74.27642496  73.47205087   0.           0.
   12.72012424]
 [ 79.05549835  67.97839046  66.76795009   0.           0.
   56.1981611 ]
 [108.63009648  70.48793568  83.44792957   0.           0.
    7.43403827]]


In [10]:
mejores_ten = np.argsort(y_papis)[:10]

limites_linea = np.array([B[i][4] for i in range(nL)])

print(papis[mejores_ten[0]], y_papis[mejores_ten[0]])

fig = px.line(x=np.arange(len(costos)), y=costos, labels={'x': 'Generación', 'y': 'Costo'}, title='Costo del despacho óptimo por generación')
fig.show()



[[47.525284   99.99874897 82.47693841  0.          0.         39.99902863]
 [39.84498953 87.22370114 62.93149494  0.          0.         79.99981438]
 [50.7540965  99.99942829 95.24650328  0.          0.         23.99997193]] 399849.1654174016


### PSO

In [16]:
import numpy as np
import random


particulas = 100
epocas = 30000


global_particulas = []  # Almacenar las posiciones actuales de la población 
global_costos = []      # Almacenar los costos actuales de las partículas 
global_costos_bloques = [] # Almacenar los costos por bloques de las partículas


# Inicialización velocidades
v_0 = np.zeros((n_bloques, nB, particulas), dtype=float) # velocidades anteriores
v_1 = np.zeros((n_bloques, nB, particulas), dtype=float) # velocidades nuevas



x_0 = None 


fit_global = None 
fit_global_costo = None


fop = np.zeros(epocas, dtype=float) 




termicas_indices = [i for i in range(nB) if G[i][1] == 2 and G[i][0] not in Gh]

# Coeficientes de costo
c = np.array([G[j][9] for j in range(nB)])
b = np.array([G[j][8] for j in range(nB)])
a = np.array([G[j][7] for j in range(nB)])

# Carga total del sistema
demanda = sum([G[i][3] for i in range(nB)])
demanda_barras = np.array([G[i][3] for i in range(nB)])

# Matriz de admitancias (Yb)
Yb = np.zeros((nB, nB), dtype=float)

for i in range(nL):
    # Asegúrate de que B[i][0] y B[i][1] son enteros y restamos 1 para índices de Python
    Yb[int(B[i][0]) - 1][int(B[i][1]) - 1] = -1 / B[i][3]
    Yb[int(B[i][1]) - 1][int(B[i][0]) - 1] = -1 / B[i][3]

for i in range(nB): # Suma de elementos en la fila, no incluyendo el diagonal
    Yb[i][i] = -np.sum(Yb[i, :]) 

# Penalización por superación de límites de líneas




while len(global_particulas) < particulas:
    posi = np.zeros((n_bloques, nB), dtype=float)
    for k in range(n_bloques):
        for j in range(nB):
            if G[j][1] == 2: 
                if G[j][0] in Gh:  # Es hidroeléctrica
                    caudal = random.uniform(0, H[k][0])
                    posi[k][j] = caudal * conver_potencia
                else:  # Es térmica
                    posi[k][j] = random.uniform(G[j][5], G[j][6])
            else:
                posi[k][j] = 0.0

    # Normalización
    suma_generaciones = np.sum(posi, axis=1)
    
    # Manejo de división por cero o sumas de generación muy pequeñas que causen problemas
    if np.any(suma_generaciones == 0) or np.any(np.abs(suma_generaciones) < 1e-6):
        continue # Descarta esta partícula y genera otra

    factores = np.array([demanda / suma_generaciones[k] for k in range(n_bloques)])
    posi_normalizada = posi * factores[:, np.newaxis]

    # Validación 
    validar_generacion_flujo = True
    
    # Validación termicas
    for k in range(n_bloques):
        for j in termicas_indices:
            if not (G[j][5] <= posi_normalizada[k, j] <= G[j][6]):
                validar_generacion_flujo = False
                break
        if not validar_generacion_flujo:
            break

    # Validación de caudal
    agua_total = 0
    if validar_generacion_flujo:
        for k in range(n_bloques):
            for barra_hidro in Gh:
                idx = barra_hidro - 1 
                caudal_hidro = posi_normalizada[k, idx] / conver_potencia
                agua_total += caudal_hidro * H[k][1] 
                if caudal_hidro > H[k][0]: 
                    validar_generacion_flujo = False
                    break
            if not validar_generacion_flujo:
                break
        
    if  agua_total > max_caudal[0]: # max_caudal[0] es el límite del embalse total
        validar_generacion_flujo = False

    # Solución de Flujo DC y Penalización por Líneas
    costo_penalizacion_bloques = np.zeros(n_bloques, dtype=float) # Penalización por bloque
    if validar_generacion_flujo:
        for k in range(n_bloques):
            try:
                xp_block = (posi_normalizada[k, :] - demanda_barras) / Sb
                # Resolver para los deltas de las barras (excepto la de referencia, índice 0)
                deltas_red = np.linalg.solve(Yb[1:, 1:], xp_block[1:])
                deltas = np.insert(deltas_red, 0, 0) # Insertar delta=0 para la barra de referencia
                
                S_flows = np.zeros(nL, dtype=float)
                for l in range(nL):
                    # Asegúrate de que los índices de B sean enteros y restamos 1
                    S_flows[l] = abs(deltas[int(B[l][0]) - 1] - deltas[int(B[l][1]) - 1]) / B[l][3]
                    S_flows[l] *= Sb # Convertir a MW
                    if not all(S_flows[l] <= B[l][4] for l in range(nL)): # si supera el límite de la línea (Smax)
                        validar_generacion_flujo = False
                        break
            except np.linalg.LinAlgError:
                validar_generacion_flujo = False # Si hay error en flujo, se considera inválido
                break # Rompe el bucle de bloques si hay error

    # Cálculo de Costo Operativo y Costo Final
    if validar_generacion_flujo:
        costo_total_individuo = 0 # Costo total del individuo (operativo + penalizaciones)
        costo_despacho_bloques_actual = np.zeros(n_bloques, dtype=float) # Costos por bloque

        for i_block in range(n_bloques):
            potencias_bloque = posi_normalizada[i_block, :]

            # Costo hidroeléctricas
            costo_hidro_bloque = 0
            for barra_hidro in Gh:
                caudal = potencias_bloque[barra_hidro - 1] / conver_potencia
                costo_hidro_bloque += caudal * H[i_block][2] # H[i][2] es el costo de oportunidad del agua

            # Costo térmicas
            costo_termicas_bloque = np.sum(c[j] * (potencias_bloque[j] ** 2) + b[j] * potencias_bloque[j] + a[j] for j in termicas_indices)

            # Sumar costos del bloque al total, incluyendo la penalización de ese bloque
            costo_despacho_bloques_actual[i_block] = costo_termicas_bloque + costo_hidro_bloque + costo_penalizacion_bloques[i_block]
            costo_total_individuo += (costo_termicas_bloque + costo_hidro_bloque) * H[i_block][1] + (costo_penalizacion_bloques[i_block] * H[i_block][1])
        
        # Si llega hasta aquí, la partícula es válida y se añade
        global_particulas.append(posi_normalizada)
        global_costos.append(costo_total_individuo)
        global_costos_bloques.append(costo_despacho_bloques_actual)




print(f"Población inicial generada con éxito: {len(global_particulas)} soluciones válidas.")

global_particulas = np.array(global_particulas)
global_costos = np.array(global_costos)
global_costos_bloques = np.array(global_costos_bloques) # Si realmente necesitas esto como array

xpb = global_particulas.copy() #mejores posiciones personales de las particulas
f_p = global_costos.copy() #mejores fitnes (costos) de las particulas

index_mejor_poblacion_inicial = np.argmin(global_costos)
fit_global_costo = global_costos[index_mejor_poblacion_inicial] #mejor costo global
fit_global = global_particulas[index_mejor_poblacion_inicial].copy() #mejor global particula



print(f"Iniciando optimización PSO para {epocas} épocas...")

for l_epoch in range(epocas): 
    

    for i_particula in range(particulas):
        w = random.uniform(0.7, 0.9) 
        C1 = random.uniform(1.5, 2.5)
        C2 = random.uniform(1.5, 2.5)
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)

        # Recuperar la información de la partícula actual
        current_xp = global_particulas[i_particula]
        current_x_pbest = xpb[i_particula]
        current_v0 = v_0[:, :, i_particula]


        v_1 = (w * current_v0 + C1 * r1 * (current_x_pbest - current_xp) + C2 * r2 * (fit_global - current_xp))

        posicion_candidata = current_xp + v_1

        # Normalizacion
        suma_generaciones_candidata = np.sum(posicion_candidata, axis=1)
        if np.any(suma_generaciones_candidata == 0) or np.any(np.abs(suma_generaciones_candidata) < 1e-6):
            continue # Descarta esta partícula si la suma es cero o muy pequeña 
        factores_candidata = np.array([demanda / suma_generaciones_candidata[k] for k in range(n_bloques)])
        posicion_final_normalizada = posicion_candidata * factores_candidata[:, np.newaxis]

        #validacion
        validar_particula = True

        #termicas
        for k in range(n_bloques):
            for j in termicas_indices:
                if not (G[j][5] <= posicion_final_normalizada[k, j] <= G[j][6]):
                    validar_particula = False
                    break
            if not validar_particula:
                break

        # caudal
        if validar_particula:
            agua_total_candidata = 0
            for k in range(n_bloques):
                for barra_hidro in Gh:
                    idx_barra = barra_hidro - 1
                    caudal_hidro = posicion_final_normalizada[k, idx_barra] / conver_potencia
                    agua_total_candidata += caudal_hidro * H[k][1]
                    if caudal_hidro > H[k][0]:
                        validar_particula = False
                        break
                if not validar_particula:
                    break
            
            if validar_particula and agua_total_candidata > max_caudal[0]:
                validar_particula = False

        # Flujo DC
        if validar_particula:
            for k in range(n_bloques):
                try:
                    xp_block_candidata = (posicion_final_normalizada[k, :] - demanda_barras) / Sb
                    deltas_red_candidata = np.linalg.solve(Yb[1:, 1:], xp_block_candidata[1:])
                    deltas_candidata = np.insert(deltas_red_candidata, 0, 0)

                    S_flows_candidata = np.zeros(nL, dtype=float)
                    for l in range(nL):
                        S_flows_candidata[l] = abs(deltas_candidata[int(B[l][0]) - 1] - deltas_candidata[int(B[l][1]) - 1]) / B[l][3]
                        S_flows_candidata[l] *= Sb
                        if not all(S_flows_candidata[l] <= B[l][4] for l in range(nL)):
                            validar_particula = False
                            break
                except np.linalg.LinAlgError:
                    validar_particula = False # Error en flujo = inválido
                    break
        
        # Cálculo del costo operativo final si todo es válido
        if validar_particula:
            costo_total = 0
            for k in range(n_bloques):
                potencias_bloque = posicion_final_normalizada[k, :]
                costo_termicas_bloque = np.sum(c[j] * (potencias_bloque[j] ** 2) + b[j] * potencias_bloque[j] + a[j] for j in termicas_indices)
                costo_hidro_bloque = 0
                for barra_hidro in Gh:
                    caudal = potencias_bloque[barra_hidro - 1] / conver_potencia
                    costo_hidro_bloque += caudal * H[k][2]
                
                costo_total += (costo_termicas_bloque + costo_hidro_bloque) * H[k][1] 
            costo_actual_particula = costo_total




        # Actualizar mejores posicion y velocidad de la particula
        if costo_actual_particula < f_p[i_particula]:
            f_p[i_particula] = costo_actual_particula
            xpb[i_particula] = posicion_final_normalizada

        # Actualizar globales
        if costo_actual_particula < fit_global_costo:
            fit_global_costo = costo_actual_particula
            fit_global = posicion_final_normalizada

        # Actualizar la posición y velocidad 
        global_particulas[i_particula] = posicion_final_normalizada
        v_0[:, :, i_particula] = v_1
        
        # Actualizar el costo de la partícula en la población principal
        global_costos[i_particula] = costo_actual_particula


    # El mejor costo por epoca
    fop[l_epoch] = fit_global_costo




C:\Users\Asus\AppData\Local\Temp\ipykernel_14828\1463377441.py:148: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_14828\1463377441.py:259: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.



Población inicial generada con éxito: 100 soluciones válidas.
Iniciando optimización PSO para 30000 épocas...


In [17]:
print("Menor costo encontrado:", fit_global_costo)
print("Mejor partícula encontrada:", fit_global)
px.line(x=np.arange(epocas), y=fop, labels={'x': 'Época', 'y': 'Costo'}, title='Costo del despacho óptimo por PSO').show()

Menor costo encontrado: 409068.8365897001
Mejor partícula encontrada: [[ 37.419648    85.09438467 108.17053948   0.           0.
   39.31542785]
 [ 36.26516347  90.18336816  63.55146837   0.           0.
   80.        ]
 [ 62.84191913  99.99759789  83.1855183    0.           0.
   23.97496468]]


### TABÚ

In [28]:



num_vecinos_por_iteracion = 250 # n vecinos por iteración
iteraciones = 30000 # n iteraciones

# Parametros tabu
tabu_duration = 10
tabu_list = []     #((idx_bloque, idx_barra_generadora), duracion_restante)


solucion_actual = None 
costo_actual_solucion = None

# mejores globales
mejor_solucion_global = None 
mejor_costo_global = None

# fop: Función objetivo de la mejor solución global en cada época.
mejores_iter = np.zeros(iteraciones, dtype=float) 


nB = len(G) 
nL = len(B) 

termicas_indices = [i for i in range(nB) if G[i][1] == 2 and G[i][0] not in Gh]
generadores_indices = [i for i in range(nB) if G[i][1] == 2] 

# coeficientes de costo
c = np.array([G[j][9] for j in range(nB)])
b = np.array([G[j][8] for j in range(nB)])
a = np.array([G[j][7] for j in range(nB)])

# demanda
demanda = sum([G[i][3] for i in range(nB)])
demanda_barras = np.array([G[i][3] for i in range(nB)])

# Yb
Yb = np.zeros((nB, nB), dtype=float)

for i in range(nL):
    Yb[int(B[i][0]) - 1][int(B[i][1]) - 1] = -1 / B[i][3]
    Yb[int(B[i][1]) - 1][int(B[i][0]) - 1] = -1 / B[i][3]

for i in range(nB): 
    Yb[i][i] = -np.sum(Yb[i, :]) 

# Penalización por superación de límites de líneas
gran_penalizacion = 1e9



while solucion_actual is None:
    posi_inicial = np.zeros((n_bloques, nB), dtype=float)
    for k in range(n_bloques):
        for j in range(nB):
            if G[j][1] == 2:  
                if G[j][0] in Gh:  
                    caudal = random.uniform(0, H[k][0])
                    posi_inicial[k][j] = caudal * conver_potencia
                else:  # Es térmica
                    posi_inicial[k][j] = random.uniform(G[j][5], G[j][6])
            else:
                posi_inicial[k][j] = 0.0

    # Normalización 
    suma_generaciones = np.sum(posi_inicial, axis=1)
    


    factores = np.array([demanda / suma_generaciones[k] for k in range(n_bloques)])
    posi_inicial_normalizada = posi_inicial * factores[:, np.newaxis]

    # Validación
    validar_solucion_inicial = True
    
    # limites termicas
    for k in range(n_bloques):
        for j in termicas_indices:
            if not (G[j][5] <= posi_inicial_normalizada[k, j] <= G[j][6]):
                validar_solucion_inicial = False
                break
        if not validar_solucion_inicial:
            break

    # Limites caudal
    if validar_solucion_inicial:
        agua_total = 0
        for k in range(n_bloques):
            for barra_hidro in Gh:
                idx = barra_hidro - 1 
                caudal_hidro = posi_inicial_normalizada[k, idx] / conver_potencia
                agua_total += caudal_hidro * H[k][1] 
                if caudal_hidro > H[k][0]: 
                    validar_solucion_inicial = False
                    break
            if not validar_solucion_inicial:
                break
        
        if validar_solucion_inicial and agua_total > max_caudal[0]: 
            validar_solucion_inicial = False

    # FLujo DC
    costo_penalizacion_bloques_inicial = np.zeros(n_bloques, dtype=float) 
    if validar_solucion_inicial:
        for k in range(n_bloques):
            try:
                xp_block = (posi_inicial_normalizada[k, :] - demanda_barras) / Sb
                deltas_red = np.linalg.solve(Yb[1:, 1:], xp_block[1:])
                deltas = np.insert(deltas_red, 0, 0) 
                
                S = np.zeros(nL, dtype=float)
                for l in range(nL):
                    S[l] = abs(deltas[int(B[l][0]) - 1] - deltas[int(B[l][1]) - 1]) / B[l][3]
                    S[l] *= Sb 
                    if not all([S[l] <= B[l][4] for l in range(nL)]):
                        validar_solucion_inicial = False
                        break 
            except np.linalg.LinAlgError:
                validar_solucion_inicial = False 
                break 

    # Costo
    if validar_solucion_inicial:
        costo_total= 0 
        for i in range(n_bloques):
            potencias_bloque = posi_inicial_normalizada[i, :]

            costo_hidro_bloque = 0
            for barra_hidro in Gh:
                caudal = potencias_bloque[barra_hidro - 1] / conver_potencia
                costo_hidro_bloque += caudal * H[i_block][2] 

            costo_termicas_bloque = np.sum(c[j] * (potencias_bloque[j] ** 2) + b[j] * potencias_bloque[j] + a[j] for j in termicas_indices)

            costo_total += (costo_termicas_bloque + costo_hidro_bloque) * H[i][1] 
        
        solucion_actual = posi_inicial_normalizada.copy()
        costo_actual_solucion = costo_total
        
        mejor_solucion_global = solucion_actual.copy()
        mejor_costo_global = costo_actual_solucion

print(f"Solución inicial generada con éxito. Costo: {mejor_costo_global:.2f}")



print(f"Iniciando Búsqueda Tabú para {iteraciones} iteraciones...")

for i in range(iteraciones): # Bucle principal de épocas
    
    vecinos_candidatos = [] # Almacena tuplas (costo, vecino_data, movimiento_generador)

    for _ in range(num_vecinos_por_iteracion):
        vecino = solucion_actual.copy()
        
        # Movimiento
        bloque_a_modificar = random.randint(0, n_bloques - 1)
        barra_a_modificar = random.choice(generadores_indices) # Solo barras con generador

        # Cambio de generación
        rango_generador = G[barra_a_modificar][6] - G[barra_a_modificar][5]
        # Si el rango es cero, no se puede generar un cambio significativo, se usa un valor por defecto.
        delta_cambio = random.uniform(-0.05, 0.05) * (rango_generador)
        
        vecino[bloque_a_modificar, barra_a_modificar] += delta_cambio

        # caracteristicas de movimiento a guardar en lista tabú
        movimiento_caracteristica = (bloque_a_modificar, barra_a_modificar)

        # Normalización del vecino
        suma_generaciones_vecino = np.sum(vecino, axis=1)
        
        if np.any(suma_generaciones_vecino == 0) or np.any((suma_generaciones_vecino) < 1e-6):
            continue # Si la suma de generaciones es cero o muy pequeña, se descarta el vecino

        factores_vecino = np.array([demanda / suma_generaciones_vecino[k] for k in range(n_bloques)])
        vecino_normalizado = vecino * factores_vecino[:, np.newaxis]

        # Validación
        validar_vecino = True
        
        # termicas
        for k in range(n_bloques):
            for j in termicas_indices:
                if not (G[j][5] <= vecino_normalizado[k, j] <= G[j][6]):
                    validar_vecino = False
                    break
            if not validar_vecino:
                break

        # caudal
        agua_total_vecino = 0
        if validar_vecino:
            for k in range(n_bloques):
                for barra_hidro_val in Gh:
                    idx = barra_hidro_val - 1
                    caudal_hidro_val = vecino_normalizado[k, idx] / conver_potencia
                    agua_total_vecino += caudal_hidro_val * H[k][1]
                    if caudal_hidro_val > H[k][0]:
                        validar_vecino = False
                        break
                if not validar_vecino:
                    break
            
        if agua_total_vecino > max_caudal[0]:
            validar_vecino = False

        # Flujo DC
        costo_penalizacion_vecino = np.zeros(n_bloques, dtype=float)
        if validar_vecino:
            for k in range(n_bloques):
                try:
                    xp = (vecino_normalizado[k, :] - demanda_barras) / Sb
                    deltas = np.linalg.solve(Yb[1:, 1:], xp[1:])
                    deltas_vecino = np.insert(deltas, 0, 0)

                    S_vecino = np.zeros(nL, dtype=float)
                    for l in range(nL):
                        S_vecino[l] = abs(deltas_vecino[int(B[l][0]) - 1] - deltas_vecino[int(B[l][1]) - 1]) / B[l][3]
                        S_vecino[l] *= Sb
                        if not all([S_vecino[l] <= B[l][4] for l in range(nL)]):
                            validar_vecino = False
                            break
                except np.linalg.LinAlgError:
                    validar_vecino = False
                    break
            
            # Costo
        if validar_vecino:
            costo_total_vecino = 0
            for k in range(n_bloques):
                potencias_bloque = vecino_normalizado[k, :]
                costo_termicas_bloque = np.sum(c[j] * (potencias_bloque[j] ** 2) + b[j] * potencias_bloque[j] + a[j] for j in termicas_indices)
                costo_hidro_bloque = 0
                for barra_hidro in Gh:
                    caudal = potencias_bloque[barra_hidro - 1] / conver_potencia
                    costo_hidro_bloque += caudal * H[k][2]
                
                costo_total_vecino += (costo_termicas_bloque + costo_hidro_bloque) * H[k][1] 
            costo_vecino = costo_total_vecino
        
            vecinos_candidatos.append((costo_vecino, vecino_normalizado.copy(), movimiento_caracteristica))

    # Mejor vecino candidato (generacion, costo, movimiento)
    best_candidate_cost = float('inf')
    best_candidate_sol = None
    best_candidate_move = None

    for costo_v, sol_v, move_v in vecinos_candidatos:
        is_tabu = False
        for tabu_entry, remaining_duration in tabu_list:
            if tabu_entry == move_v: # Si el movimiento está en la lista tabú
                is_tabu = True
                break
        
        # Si se encontro dentro del tabu, verifica que el costo sea mejor que el global
        if is_tabu and costo_v < mejor_costo_global:
            is_tabu = False 

        if not is_tabu:
            if costo_v < best_candidate_cost:
                best_candidate_cost = costo_v
                best_candidate_sol = sol_v
                best_candidate_move = move_v
    
    if best_candidate_sol is not None:
        solucion_actual = best_candidate_sol.copy()
        costo_actual_solucion = best_candidate_cost
        
        # Añadir el movimiento que llevó a esta nueva solución a la lista tabú
        tabu_list.append((best_candidate_move, tabu_duration))

    # Actualizar lista y decrementar duración de movimientos en la lista tabú
    new_tabu_list = []
    for move_entry, duration in tabu_list:
        if duration > 1:
            new_tabu_list.append((move_entry, duration - 1))
    tabu_list = new_tabu_list

    # Actualizar mejor solucion global
    if costo_actual_solucion < mejor_costo_global:
        mejor_costo_global = costo_actual_solucion
        mejor_solucion_global = solucion_actual.copy()
    
    # Almacenar el mejor costo de esta iteracion
    mejores_iter[i] = mejor_costo_global

print(f"\nBúsqueda Tabú terminada. Mejor costo encontrado: {mejor_costo_global:.2f}")
print("Mejor posición global (generación por bloque y barra):")
print(mejor_solucion_global)

C:\Users\Asus\AppData\Local\Temp\ipykernel_14828\4216481775.py:131: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

C:\Users\Asus\AppData\Local\Temp\ipykernel_14828\4216481775.py:231: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.



Solución inicial generada con éxito. Costo: 619524.66
Iniciando Búsqueda Tabú para 30000 iteraciones...

Búsqueda Tabú terminada. Mejor costo encontrado: 399279.15
Mejor posición global (generación por bloque y barra):
[[51.40093647 99.99181048 78.62382215  0.          0.         39.98343089]
 [34.47128255 93.17627566 62.35732686  0.          0.         79.99511492]
 [51.79882921 99.98753631 94.24142645  0.          0.         23.97220803]]


In [29]:
print(f"Mejor costo global: {mejor_costo_global:.2f}")
print(f"Mejor solución global: {mejor_solucion_global}")

px.line(x=np.arange(iteraciones), y=mejores_iter, labels={'x': 'Iteración', 'y': 'Costo'}, title='Costo del despacho óptimo por Búsqueda Tabú').show()

Mejor costo global: 399279.15
Mejor solución global: [[51.40093647 99.99181048 78.62382215  0.          0.         39.98343089]
 [34.47128255 93.17627566 62.35732686  0.          0.         79.99511492]
 [51.79882921 99.98753631 94.24142645  0.          0.         23.97220803]]


In [30]:
#grafica global
df_global = pd.DataFrame({'iteración':np.arange(iteraciones), 'Búsqueda Tabú':mejores_iter, 'PSO':fop, 'Computación Genética':costos})
fig = px.line(df_global, x='iteración', y=['Búsqueda Tabú', 'PSO', 'Computación Genética'], title='Comparación de costos entre Búsqueda Tabú, PSO y Algoritmo Genético', labels={'value': 'Costo', 'variable': 'Método'})
fig.show()